In [1]:
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install -q git+https://github.com/rusty1s/pytorch_geometric.git

     |████████████████████████████████| 8.0 MB 4.8 MB/s 
     |████████████████████████████████| 2.9 MB 5.3 MB/s 
     |████████████████████████████████| 376 kB 5.2 MB/s 
     |████████████████████████████████| 45 kB 3.3 MB/s 


In [2]:
import torch_geometric
from torch_geometric.datasets import Planetoid

In [3]:
use_cuda_if_available = False

In [4]:
dataset = Planetoid(root="tutorial1",name= "Cora")

Processing...
Done!


In [5]:
print(dataset)
print("number of graphs:\t\t",len(dataset))
print("number of classes:\t\t",dataset.num_classes)
print("number of node features:\t",dataset.num_node_features)
print("number of edge features:\t",dataset.num_edge_features)

Cora()
number of graphs:		 1
number of classes:		 7
number of node features:	 1433
number of edge features:	 0


In [6]:
print(dataset.data)

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [7]:
print("edge_index:\t\t",dataset.data.edge_index.shape)
print(dataset.data.edge_index)
print("-"*50)
print("train_mask:\t\t",dataset.data.train_mask.shape)
print(dataset.data.train_mask)
print("-"*50)
print("x(data (nodes x features)):\t\t",dataset.data.x.shape)
print(dataset.data.x)
print("-"*50)
print("y(label):\t\t",dataset.data.y.shape)
print(dataset.data.y)

edge_index:		 torch.Size([2, 10556])
tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
        [ 633, 1862, 2582,  ...,  598, 1473, 2706]])
--------------------------------------------------
train_mask:		 torch.Size([2708])
tensor([ True,  True,  True,  ..., False, False, False])
--------------------------------------------------
x(data (nodes x features)):		 torch.Size([2708, 1433])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
--------------------------------------------------
y(label):		 torch.Size([2708])
tensor([3, 4, 4,  ..., 3, 3, 3])


In [8]:
import os.path as osp

import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv

In [9]:
data = dataset[0] #because we only have one dataset

In [10]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv = SAGEConv(dataset.num_features,
                             dataset.num_classes,
                             aggr="max") # max, mean, add ...)

    def forward(self):
        x = self.conv(data.x, data.edge_index)
        return F.log_softmax(x, dim=1)

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() and use_cuda_if_available else 'cpu')
model, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [15]:
def train():
    model.train()
    optimizer.zero_grad()
    F.nll_loss(model()[data.train_mask], data.y[data.train_mask]).backward()
    optimizer.step()


def test():
    model.eval()
    logits, accs = model(), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

In [14]:
def get_gpu_details():
    t = torch.cuda.get_device_properties(0).total_memory
    c = torch.cuda.memory_cached(0)
    a = torch.cuda.memory_allocated(0)
    print(torch.cuda.get_device_name())
    print(f'Total GPU Memory {t} B , Cached GPU Memory {c} B, Allocated GPU Memory {a} B')
    
    
    
if torch.cuda.is_available():
    device='cuda'
else:
    device='cpu'
print(f'Current Device: {device}')
if device=='cuda':
    torch.cuda.empty_cache()
    get_gpu_details()

Current Device: cuda
Tesla K80
Total GPU Memory 11996954624 B , Cached GPU Memory 0 B, Allocated GPU Memory 0 B


/usr/local/lib/python3.7/dist-packages/torch/cuda/memory.py:375: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  FutureWarning)


In [18]:
best_val_acc = test_acc = 0
for epoch in range(1,100):
    train()
    _, val_acc, tmp_test_acc = test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    log = 'Epoch: {:03d}, Val: {:.4f}, Test: {:.4f}'
    
    if epoch % 10 == 0:
        print(log.format(epoch, best_val_acc, test_acc))

Epoch: 010, Val: 0.7320, Test: 0.7240
Epoch: 020, Val: 0.7320, Test: 0.7240
Epoch: 030, Val: 0.7320, Test: 0.7240
Epoch: 040, Val: 0.7320, Test: 0.7240
Epoch: 050, Val: 0.7320, Test: 0.7240
Epoch: 060, Val: 0.7320, Test: 0.7240
Epoch: 070, Val: 0.7320, Test: 0.7240
Epoch: 080, Val: 0.7320, Test: 0.7240
Epoch: 090, Val: 0.7320, Test: 0.7240
